##  Лабораторна робота No4

### Структури для роботи з великими обсягами даних в Python
#### ФБ-21 Худоба Арсен
<b>Мета:</b> отримати навички роботи із структурами для зберігання в Python (python, numpy, pandas, numpy array, dataframe, timeit)
Основні поняття: numpy масиви, кортежі, списки, фрейми, профілювання.
### Перший рівень (спрощені завдання)


In [25]:
import os
import pandas as pd 
from ucimlrepo import fetch_ucirepo
import numpy as np
import time

In [230]:
# fetch dataset 
individual_household_electric_power_consumption = fetch_ucirepo(id=235) 
  
# data (as pandas dataframes) 
X = individual_household_electric_power_consumption.data.features 
y = individual_household_electric_power_consumption.data.targets 
  
# metadata 
print(individual_household_electric_power_consumption.metadata) 
  
# variable information 
print(individual_household_electric_power_consumption.variables) 


{'uci_id': 235, 'name': 'Individual Household Electric Power Consumption', 'repository_url': 'https://archive.ics.uci.edu/dataset/235/individual+household+electric+power+consumption', 'data_url': 'https://archive.ics.uci.edu/static/public/235/data.csv', 'abstract': 'Measurements of electric power consumption in one household with a one-minute sampling rate over a period of almost 4 years. Different electrical quantities and some sub-metering values are available.', 'area': 'Physics and Chemistry', 'tasks': ['Regression', 'Clustering'], 'characteristics': ['Multivariate', 'Time-Series'], 'num_instances': 2075259, 'num_features': 9, 'feature_types': ['Real'], 'demographics': [], 'target_col': None, 'index_col': None, 'has_missing_values': 'no', 'missing_values_symbol': None, 'year_of_dataset_creation': 2006, 'last_updated': 'Fri Mar 08 2024', 'dataset_doi': '10.24432/C58K54', 'creators': ['Georges Hebrail', 'Alice Berard'], 'intro_paper': None, 'additional_info': {'summary': 'This archiv

C:\Users\Arsen\AppData\Roaming\Python\Python311\site-packages\ucimlrepo\fetch.py:97: DtypeWarning: Columns (2,3,4,5,6,7) have mixed types. Specify dtype option on import or set low_memory=False.
  df = pd.read_csv(data_url)


In [231]:
X

,Date,Time,Global_active_power,Global_reactive_power,Voltage,Global_intensity,Sub_metering_1,Sub_metering_2,Sub_metering_3
0,16/12/2006,17:24:00,4.216,0.418,234.840,18.400,0.000,1.000,17.0
1,16/12/2006,17:25:00,5.360,0.436,233.630,23.000,0.000,1.000,16.0
2,16/12/2006,17:26:00,5.374,0.498,233.290,23.000,0.000,2.000,17.0
3,16/12/2006,17:27:00,5.388,0.502,233.740,23.000,0.000,1.000,17.0
4,16/12/2006,17:28:00,3.666,0.528,235.680,15.800,0.000,1.000,17.0
...,...,...,...,...,...,...,...,...,...
2075254,26/11/2010,20:58:00,0.946,0.0,240.43,4.0,0.0,0.0,0.0
2075255,26/11/2010,20:59:00,0.944,0.0,240.0,4.0,0.0,0.0,0.0
2075256,26/11/2010,21:00:00,0.938,0.0,239.82,3.8,0.0,0.0,0.0
2075257,26/11/2010,21:01:00,0.934,0.0,239.7,3.8,0.0,0.0,0.0


Приберемо пусті значення

In [26]:
file_path = 'household_power_consumption.txt'
df = pd.read_csv(file_path, delimiter=';', dtype={'Date': str, 'Time': str, 'Global_active_power': float, 'Global_reactive_power': float, 'Voltage': float, 'Global_intensity': float, 'Sub_metering_1': float, 'Sub_metering_2': float, 'Sub_metering_3': float}, na_values=['?'])
df = df.dropna()
# Перейменовуємо стовпці
df = df.rename(columns={
    'Date': 'Date',
    'Time': 'Time',
    'Global_active_power': 'ActiveP',
    'Global_reactive_power': 'ReactiveP',
    'Voltage': 'Voltage',
    'Global_intensity': 'Intensity',
    'Sub_metering_1': 'Sub1',
    'Sub_metering_2': 'Sub2',
    'Sub_metering_3': 'Sub3'
})

# Виводимо перші декілька рядків DataFrame для перевірки
df.head()

,Date,Time,ActiveP,ReactiveP,Voltage,Intensity,Sub1,Sub2,Sub3
0,16/12/2006,17:24:00,4.216,0.418,234.84,18.4,0.0,1.0,17.0
1,16/12/2006,17:25:00,5.360,0.436,233.63,23.0,0.0,1.0,16.0
2,16/12/2006,17:26:00,5.374,0.498,233.29,23.0,0.0,2.0,17.0
3,16/12/2006,17:27:00,5.388,0.502,233.74,23.0,0.0,1.0,17.0
4,16/12/2006,17:28:00,3.666,0.528,235.68,15.8,0.0,1.0,17.0


1. Обрати всі домогосподарства, у яких загальна активна споживана потужність перевищує 5 кВт.

In [27]:
start = time.time()
selected_households = df[df['ActiveP'] > 5.0]
end = time.time()
print(f'Used time PANDAS:{end-start}')
selected_households.head()

Used time PANDAS:0.00651097297668457


,Date,Time,ActiveP,ReactiveP,Voltage,Intensity,Sub1,Sub2,Sub3
1,16/12/2006,17:25:00,5.360,0.436,233.63,23.0,0.0,1.0,16.0
2,16/12/2006,17:26:00,5.374,0.498,233.29,23.0,0.0,2.0,17.0
3,16/12/2006,17:27:00,5.388,0.502,233.74,23.0,0.0,1.0,17.0
11,16/12/2006,17:35:00,5.412,0.470,232.78,23.2,0.0,1.0,17.0
12,16/12/2006,17:36:00,5.224,0.478,232.99,22.4,0.0,1.0,16.0


In [28]:
start = time.time()

filtered_df = df[np.where(df['ActiveP'] > 5.0, True, False)]

end = time.time()

print(f'Used time NUMPY:{end-start}')
filtered_df.head()

Used time NUMPY:0.005606412887573242


,Date,Time,ActiveP,ReactiveP,Voltage,Intensity,Sub1,Sub2,Sub3
1,16/12/2006,17:25:00,5.360,0.436,233.63,23.0,0.0,1.0,16.0
2,16/12/2006,17:26:00,5.374,0.498,233.29,23.0,0.0,2.0,17.0
3,16/12/2006,17:27:00,5.388,0.502,233.74,23.0,0.0,1.0,17.0
11,16/12/2006,17:35:00,5.412,0.470,232.78,23.2,0.0,1.0,17.0
12,16/12/2006,17:36:00,5.224,0.478,232.99,22.4,0.0,1.0,16.0


2. Обрати всі домогосподарства, у яких вольтаж перевищую 235 В.

In [29]:
selected_households = df[df['Voltage'] > 235.0]
selected_households.head()

,Date,Time,ActiveP,ReactiveP,Voltage,Intensity,Sub1,Sub2,Sub3
4,16/12/2006,17:28:00,3.666,0.528,235.68,15.8,0.0,1.0,17.0
5,16/12/2006,17:29:00,3.520,0.522,235.02,15.0,0.0,2.0,17.0
6,16/12/2006,17:30:00,3.702,0.520,235.09,15.8,0.0,1.0,17.0
7,16/12/2006,17:31:00,3.700,0.520,235.22,15.8,0.0,1.0,17.0
14,16/12/2006,17:38:00,4.054,0.422,235.24,17.6,0.0,1.0,17.0


3. Обрати всі домогосподарства, у яких сила струму лежить в межах 19-20 А, для них виявити ті, у яких пральна машина та холодильних споживають більше, ніж бойлер та кондиціонер.

In [30]:
start = time.time()

selected_households = df[
    (df['Intensity'] >= 19.0) & 
    (df['Intensity'] <= 20.0) & 
    (df['Sub2'] > df['Sub3'])
]

end = time.time()

print(f'Used time PANDAS:{end-start}')
selected_households.head()

Used time PANDAS:0.012141942977905273


,Date,Time,ActiveP,ReactiveP,Voltage,Intensity,Sub1,Sub2,Sub3
45,16/12/2006,18:09:00,4.464,0.136,234.66,19.0,0.0,37.0,16.0
460,17/12/2006,01:04:00,4.582,0.258,238.08,19.6,0.0,13.0,0.0
464,17/12/2006,01:08:00,4.618,0.104,239.61,19.6,0.0,27.0,0.0
475,17/12/2006,01:19:00,4.636,0.140,237.37,19.4,0.0,36.0,0.0
476,17/12/2006,01:20:00,4.634,0.152,237.17,19.4,0.0,35.0,0.0


In [31]:
start = time.time()

filtered_df = df[np.where(
    (df['Intensity'] >= 19.0) & 
    (df['Intensity'] <= 20.0) & 
    (df['Sub2'] > df['Sub3']), True, False
)]

end = time.time()

print(f'Used time NUMPY:{end-start}')
filtered_df.head()

Used time NUMPY:0.012531757354736328


,Date,Time,ActiveP,ReactiveP,Voltage,Intensity,Sub1,Sub2,Sub3
45,16/12/2006,18:09:00,4.464,0.136,234.66,19.0,0.0,37.0,16.0
460,17/12/2006,01:04:00,4.582,0.258,238.08,19.6,0.0,13.0,0.0
464,17/12/2006,01:08:00,4.618,0.104,239.61,19.6,0.0,27.0,0.0
475,17/12/2006,01:19:00,4.636,0.140,237.37,19.4,0.0,36.0,0.0
476,17/12/2006,01:20:00,4.634,0.152,237.17,19.4,0.0,35.0,0.0


4. Обрати випадковим чином 500000 домогосподарств (без повторів елементів вибірки), для них обчислити середні величини усіх 3-х груп споживання електричної енергії

In [32]:
random_indexes = np.random.choice(df.index, size=500000, replace=True)
random_sample = df.loc[random_indexes]

average_consumption = random_sample[['Sub1', 'Sub2', 'Sub3']].mean()

print("Середні величини усіх 3-х груп споживання електричної енергії:", average_consumption)

Середні величини усіх 3-х груп споживання електричної енергії: Sub1    1.128874
Sub2    1.310464
Sub3    6.482480
dtype: float64


5. Обрати ті домогосподарства, які після 18-00 споживають понад 6 кВт за хвилину в середньому, серед відібраних визначити ті, у яких основне споживання електроенергії у вказаний проміжок часу припадає на пральну машину, сушарку, холодильник та освітлення(група 2 є найбільшою), а потім обрати кожен третій результат із першої половини та кожен четвертий результат із другої половини.

In [33]:
start = time.time()

high_consumption = df[df['Time'] > '18:00']
main_consumption = high_consumption[(high_consumption['ActiveP'] > 6.0) &
                                    (high_consumption['Sub2'] > high_consumption['Sub1']) & 
                                    (high_consumption['Sub2'] > high_consumption['Sub3'])]
end = time.time()

selected_households = pd.concat([main_consumption.loc[::3, :], main_consumption.iloc[::4, :]])

print(f'Used time PANDAS:{end-start}')

selected_households.head()

Used time PANDAS:0.0922856330871582


,Date,Time,ActiveP,ReactiveP,Voltage,Intensity,Sub1,Sub2,Sub3
41,16/12/2006,18:05:00,6.052,0.192,232.93,26.2,0.0,37.0,17.0
44,16/12/2006,18:08:00,6.308,0.116,232.25,27.0,0.0,36.0,17.0
17494,28/12/2006,20:58:00,6.386,0.374,236.63,27.0,1.0,36.0,17.0
17498,28/12/2006,21:02:00,8.088,0.262,235.50,34.4,1.0,72.0,17.0
17501,28/12/2006,21:05:00,7.230,0.152,235.22,30.6,1.0,73.0,17.0


In [34]:
start = time.time()

high_consumption = df[df['Time'] > '18:00']
main_consumption_mask = (high_consumption['ActiveP'] > 6.0) & (high_consumption['Sub2'] > high_consumption['Sub1']) & (high_consumption['Sub2'] > high_consumption['Sub3'])

main_consumption = high_consumption[np.where(main_consumption_mask, True, False)]

end = time.time()

selected_households = pd.concat([main_consumption.loc[::3, :], main_consumption.iloc[::4, :]])

print(f'Used time NUMPY:{end-start}')

selected_households.head()

Used time NUMPY:0.09243226051330566


,Date,Time,ActiveP,ReactiveP,Voltage,Intensity,Sub1,Sub2,Sub3
41,16/12/2006,18:05:00,6.052,0.192,232.93,26.2,0.0,37.0,17.0
44,16/12/2006,18:08:00,6.308,0.116,232.25,27.0,0.0,36.0,17.0
17494,28/12/2006,20:58:00,6.386,0.374,236.63,27.0,1.0,36.0,17.0
17498,28/12/2006,21:02:00,8.088,0.262,235.50,34.4,1.0,72.0,17.0
17501,28/12/2006,21:05:00,7.230,0.152,235.22,30.6,1.0,73.0,17.0
